In [207]:
import shutil

#shutil.rmtree("/content/Fresh_and_Rotten_Classification")
#shutil.rmtree("/content/Fruit_And_Vegetable_Diseases_Dataset")


In [208]:
from google.colab import drive
import zipfile
import os

# Google Drive'ı bağla
drive.mount('/content/drive')

# Dataset 1: Fresh_and_Rotten_Classification
zip_path1 = "/content/drive/MyDrive/Colab Notebooks/Fresh_and_Rotten_Classification.zip"
extract_path1 = "/content/Fresh_and_Rotten_Classification"

if not os.path.exists(extract_path1):
    with zipfile.ZipFile(zip_path1, 'r') as zip_ref:
        zip_ref.extractall(extract_path1)
    print(f"dataset1 çıkartıldı: {extract_path1}")
else:
    print(f"dataset1 zaten mevcut, tekrar çıkartılmadı: {extract_path1}")

# Dataset 2: Fruit_And_Vegetable_Diseases_Dataset
zip_path2 = "/content/drive/MyDrive/Colab Notebooks/Fruit_And_Vegetable_Diseases_Dataset.zip"
extract_path2 = "/content/Fruit_And_Vegetable_Diseases_Dataset"

if not os.path.exists(extract_path2):
    with zipfile.ZipFile(zip_path2, 'r') as zip_ref:
        zip_ref.extractall(extract_path2)
    print(f"dataset2 çıkartıldı: {extract_path2}")
else:
    print(f"dataset2 zaten mevcut, tekrar çıkartılmadı: {extract_path2}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
dataset1 zaten mevcut, tekrar çıkartılmadı: /content/Fresh_and_Rotten_Classification
dataset2 zaten mevcut, tekrar çıkartılmadı: /content/Fruit_And_Vegetable_Diseases_Dataset


In [209]:
import tensorflow as tf

def find_tf_bozuk_gorseller(root_dir):
    bozuklar = []

    for class_dir in os.listdir(root_dir):
        class_path = os.path.join(root_dir, class_dir)
        if not os.path.isdir(class_path):
            continue

        for file in os.listdir(class_path):
            file_path = os.path.join(class_path, file)
            try:
                img_raw = tf.io.read_file(file_path)
                _ = tf.image.decode_image(img_raw)  # TensorFlow'un kendi decoder'ı
            except Exception as e:
                print(f"TF bozuk: {file_path} - {e}")
                bozuklar.append(file_path)

    print(f"\nTensorFlow tarafından okunamayan dosya sayısı: {len(bozuklar)}")
    for f in bozuklar:
      try:
          os.remove(f)
          print(f"Silindi: {f}")
      except Exception as e:
          print(f"Silinemedi: {f} - {e}")

    return bozuklar

#bozuklar = find_tf_bozuk_gorseller("/content/Fruit_And_Vegetable_Diseases_Dataset/Fruit_And_Vegetable_Diseases_Dataset/Test")
#bozuklar = find_tf_bozuk_gorseller("/content/Fruit_And_Vegetable_Diseases_Dataset/Fruit_And_Vegetable_Diseases_Dataset/Train")


Üstteki 3 hücredeki kodu chatgpt den yazdırdım çünkü google colab üzerinden çalıştığımdan

ilk hücre, dosyaları silmeme yarıyor yoksa colab üzerinden boş olmayan dosyayı silmeme izin vermiyor bu yüzden kod ile çözdürdüm.

ikinci hücre, dosya sistemi colab üzerinde biraz daha karmaşık olduğundan ve veri setlerimi drive'a yüklerken ziplediğimden gerekiyor.

üçüncü hücre de bozuk olan görselleri (.webp ve boş uzantılı şimdilik denk gelinen dosyalar) tespit edip siliyor.


In [210]:
import os
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import SGD

BATCH_SIZE = 128
EPOCHS = 50


In [211]:
IMAGE_SIZE = (224, 224)
SEED = 41

train_dir_1 = "/content/Fresh_and_Rotten_Classification/Fresh_and_Rotten_Classification/Train"
test_dir_1  = "/content/Fresh_and_Rotten_Classification/Fresh_and_Rotten_Classification/Test"

train_dir_2 = "/content/Fruit_And_Vegetable_Diseases_Dataset/Fruit_And_Vegetable_Diseases_Dataset/Train"
test_dir_2  = "/content/Fruit_And_Vegetable_Diseases_Dataset/Fruit_And_Vegetable_Diseases_Dataset/Test"

In [212]:
# Dataset 1: Fresh and Rotten
train_ds_1 = image_dataset_from_directory(
    train_dir_1,
    validation_split=0.2,
    subset="training",
    seed=SEED,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True
)

val_ds_1 = image_dataset_from_directory(
    train_dir_1,
    validation_split=0.2,
    subset="validation",
    seed=SEED,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True
)

test_ds_1 = image_dataset_from_directory(
    test_dir_1,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False
)


Found 21044 files belonging to 14 classes.
Using 16836 files for training.
Found 21044 files belonging to 14 classes.
Using 4208 files for validation.
Found 6738 files belonging to 14 classes.


In [213]:
# Dataset 2: Fruit and Vegetable Disease
train_ds_2 = image_dataset_from_directory(
    train_dir_2,
    validation_split=0.2,
    subset="training",
    seed=SEED,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True
)

val_ds_2 = image_dataset_from_directory(
    train_dir_2,
    validation_split=0.2,
    subset="validation",
    seed=SEED,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True
)

test_ds_2 = image_dataset_from_directory(
    test_dir_2,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False
)

Found 23425 files belonging to 28 classes.
Using 18740 files for training.
Found 23425 files belonging to 28 classes.
Using 4685 files for validation.
Found 5849 files belonging to 28 classes.


In [214]:
# .class_names kaybolmadan önce bu değerleri kaydediyoruz
class_names_1 = train_ds_1.class_names
num_classes_1 = len(class_names_1)
test_class_names_1 = test_ds_1.class_names
test_num_classes_1 = len(test_class_names_1)

class_names_2 = train_ds_2.class_names
num_classes_2 = len(class_names_2)
test_class_names_2 = test_ds_2.class_names
test_num_classes_2 = len(test_class_names_2)


print("Fresh_and_Rotten sınıfları:", class_names_1)
print("Sınıf sayısı:", num_classes_1)
print("Test sınıfları:", test_class_names_1)
print("Test sınıf sayısı:", test_num_classes_1)


print("Fruit_and_Veg_Disease sınıfları:", class_names_2)
print("Sınıf sayısı:", num_classes_2)
print("Test sınıfları:", test_class_names_2)
print("Test sınıf sayısı:", test_num_classes_2)



Fresh_and_Rotten sınıfları: ['freshapples', 'freshbanana', 'freshcucumber', 'freshokra', 'freshoranges', 'freshpotato', 'freshtomato', 'rottenapples', 'rottenbanana', 'rottencucumber', 'rottenokra', 'rottenoranges', 'rottenpotato', 'rottentomato']
Sınıf sayısı: 14
Test sınıfları: ['freshapples', 'freshbanana', 'freshcucumber', 'freshokra', 'freshoranges', 'freshpotato', 'freshtomato', 'rottenapples', 'rottenbanana', 'rottencucumber', 'rottenokra', 'rottenoranges', 'rottenpotato', 'rottentomato']
Test sınıf sayısı: 14
Fruit_and_Veg_Disease sınıfları: ['Apple__Healthy', 'Apple__Rotten', 'Banana__Healthy', 'Banana__Rotten', 'Bellpepper__Healthy', 'Bellpepper__Rotten', 'Carrot__Healthy', 'Carrot__Rotten', 'Cucumber__Healthy', 'Cucumber__Rotten', 'Grape__Healthy', 'Grape__Rotten', 'Guava__Healthy', 'Guava__Rotten', 'Jujube__Healthy', 'Jujube__Rotten', 'Mango__Healthy', 'Mango__Rotten', 'Orange__Healthy', 'Orange__Rotten', 'Pomegranate__Healthy', 'Pomegranate__Rotten', 'Potato__Healthy', 'Po

In [215]:
def normalize_images(image, label):
    image = image / 255.0
    return image, label

# 1. Dataset
train_ds_1 = train_ds_1.map(normalize_images)
val_ds_1 = val_ds_1.map(normalize_images)
test_ds_1 = test_ds_1.map(normalize_images)

# 2. Dataset
train_ds_2 = train_ds_2.map(normalize_images)
val_ds_2 = val_ds_2.map(normalize_images)
test_ds_2 = test_ds_2.map(normalize_images)


In [216]:
# CNN Modeli
def create_cnn_model(num_classes):
    model = Sequential()

    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
    model.add(MaxPooling2D())

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D())

    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D())

    model.add(Flatten())
    model.add(Dense(192, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    optimizer = SGD(learning_rate=0.01)
    model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [217]:
os.makedirs("saved_models", exist_ok=True)
#"Fresh_and_Rotten": (train_ds_1, val_ds_1, test_ds_1, num_classes_1),
#"Fruit_And_Veg_Disease": (train_ds_2, val_ds_2, test_ds_2, num_classes_2)
datasets = {
    "Fresh_and_Rotten": (train_ds_1, val_ds_1, test_ds_1, num_classes_1),
    "Fruit_And_Veg_Disease": (train_ds_2, val_ds_2, test_ds_2, num_classes_2)
}


for name, (train_ds, val_ds, test_ds, num_classes) in datasets.items():
    print(f"\nEğitim Başladı - {name}")

    model = create_cnn_model(num_classes)

    checkpoint_path = f"saved_models/cnn_model_{name}.keras"

    callbacks = [
        EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
        ModelCheckpoint(filepath=checkpoint_path, monitor='val_accuracy', save_best_only=True)
    ]

    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=EPOCHS,
        callbacks=callbacks
    )

    loss, acc = model.evaluate(test_ds)
    print(f"{name} için Test Doğruluğu: {acc:.4f}")




Eğitim Başladı - Fresh_and_Rotten
Epoch 1/50
132/132 ━━━━━━━━━━━━━━━━━━━━ 12s 73ms/step - accuracy: 0.2468 - loss: 2.2696 - val_accuracy: 0.4881 - val_loss: 1.3836
Epoch 2/50
132/132 ━━━━━━━━━━━━━━━━━━━━ 8s 63ms/step - accuracy: 0.5117 - loss: 1.3552 - val_accuracy: 0.5288 - val_loss: 1.4478
Epoch 3/50
132/132 ━━━━━━━━━━━━━━━━━━━━ 8s 63ms/step - accuracy: 0.6210 - loss: 1.0188 - val_accuracy: 0.6457 - val_loss: 0.8962
Epoch 4/50
132/132 ━━━━━━━━━━━━━━━━━━━━ 8s 64ms/step - accuracy: 0.7110 - loss: 0.7814 - val_accuracy: 0.7341 - val_loss: 0.6889
Epoch 5/50
132/132 ━━━━━━━━━━━━━━━━━━━━ 8s 62ms/step - accuracy: 0.7576 - loss: 0.6566 - val_accuracy: 0.7369 - val_loss: 0.7324
Epoch 6/50
132/132 ━━━━━━━━━━━━━━━━━━━━ 9s 64ms/step - accuracy: 0.7806 - loss: 0.5836 - val_accuracy: 0.7902 - val_loss: 0.5702
Epoch 7/50
132/132 ━━━━━━━━━━━━━━━━━━━━ 8s 62ms/step - accuracy: 0.8113 - loss: 0.5068 - val_accuracy: 0.8127 - val_loss: 0.4988
Epoch 8/50
132/132 ━━━━━━━━━━━━━━━━━━━━ 8s 64ms/step - accura